In [8]:
import numpy as np
import math
import os
import random
import json
from scipy.io import loadmat
from pathlib import Path

In [9]:
# scan for PartNet chair models
partNet_path = 'data/PartNet_Chairs/Chair_parts/'
models = [x.name for x in os.scandir(partNet_path)]
print(len(models), 'chair models found.')

6201 chair models found.


In [12]:
# This helper function appends OBJ file content (vertices and faces) to specified output
# vert_offset is used to correct face-to-vertex references
def appendOBJ(obj_content, out_file, vert_offset):
    new_vert_offset = vert_offset
    for line in obj_content:
        if line.startswith('v '):
            out_file.write(line)
            new_vert_offset += 1
        elif line.startswith('f '):
            params = line.split(' ')
            v1 = int(params[1])
            v2 = int(params[2])
            v3 = int(params[3])
            out_file.write('f %d %d %d\n' % (v1+vert_offset, v2+vert_offset, v3+vert_offset))
    return new_vert_offset

In [14]:
# open output OBJ file for writing
Path('out/').mkdir(parents=True, exist_ok=True)
output_filename = 'out/output.obj'
output_obj_file = open(output_filename, 'w')

# offset used to correct vertex indexing in output file
vert_offset_global = 0

for label in ['chair_base', 'chair_back', 'chair_seat', 'chair_arm']:
    # pick a random chair
    model_idx = random.choice(models)
    print('Using %s from chair %s' % (label, model_idx))
    
    # open obj file for writing (current part only)
    output_obj_part_file = open('out/' + label + '.obj', 'w')
    
    # offset used to correct vertex indexing in part output file
    vert_offset_part = 0
    
    # open JSON hierarchy from PartNet model and extract OBJ file names for current label
    json_file = open(partNet_path + model_idx + '/result_after_merging.json', 'r')
    json_content = json.load(json_file)
    json_file.close()
    obj_lists = [child['objs'] for child in json_content[0]['children'] if child['name'] == label]
    if len(obj_lists) == 0:
        print('No %s found in chair %s' % (label, model_idx))
        
    # append OBJ file content to part OBJ output and total OBJ output
    for obj_list in obj_lists:
        for obj_name in obj_list:
            with open(partNet_path + model_idx + '/objs/' + obj_name + '.obj', 'r') as obj_file:
                obj_lines = [line for line in obj_file]
                vert_offset_global = appendOBJ(obj_lines, output_obj_file, vert_offset_global)
                vert_offset_part = appendOBJ(obj_lines, output_obj_part_file, vert_offset_part)
    output_obj_part_file.close()

# close output file
output_obj_file.close()
print('Output saved to file', output_filename)

Using chair_base from chair 36328
Using chair_back from chair 40750
Using chair_seat from chair 38758
Using chair_arm from chair 36908
Output saved to file out/output.obj
